In [1]:
%matplotlib widget
''' 系统库 '''
import datetime
import time
from random import randint 
from importlib import reload 
# from numba import jit
# @jit(nopython=True)

import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt

''' 业务库 '''
# import mplfinance as mpf
import tushare as ts
ts.set_token('11137efdbeac800606d677871a55b3fd5aef79890c59867a8f34d03e')
pro = ts.pro_api()

''' 自建库 '''
from qflib import basic
reload(basic)

global engine, conn 
engine = basic.engine()
conn = basic.conn(engine)

trade_date = datetime.datetime.now().strftime('%Y%m%d')
print('date : ', trade_date)
run_static, run_dynamic = False, True
# run_static, run_dynamic = True, True
# run_static, run_dynamic = True, False


date :  20230725


In [2]:
# 获得 bak_basic
def sync_basic(conn, df, table, trade_date):    
    sql = " truncate table "+table
    print( table, basic.exec_sql(conn, sql) )
    basic.write_data(conn, df, table, False, 'append')

#取000001的前复权行情
#df = ts.pro_bar(ts_code='000001.SZ', adj='qfq', start_date='20180101', end_date='20181011')
#取000001的后复权行情
#df = ts.pro_bar(ts_code='000001.SZ', adj='hfq', start_date='20180101', end_date='20181011')
# 分钟数据，freq/数据频度 ：1MIN表示1分钟（1/5/15/30/60分钟） D日线 ，默认D， W，M;  *** 您每天最多访问该接口2次
# df = ts.pro_bar(ts_code='601318.SH', adj='qfq', freq='1min', start_date='20220401', end_date='20220430')
# print(df.head(3))
# df.to_csv('../data/minite_601318.csv')
# df.info
# df['hs_type'].unique(), df['hs_type'].value_counts(), df.hs_type.value_counts()

In [3]:

# df = pro.stock_basic(exchange='', list_status='L')
# df = pro.concept(src='ts') 
# sync_basic(engine, df, 'concept', trade_date)
# df = pro.hs_const(hs_type='SH')

In [4]:

# write_no = df.to_sql('sl_hugutong', conn, index=False, if_exists='append', chunksize=2000) # 每次写入2000个

In [5]:
%%time
# 静态数据更新  -  调整为手工处理
if run_static:
    print( '更新基本信息' )
    df = pro.stock_basic(exchange='', list_status='L')
    sync_basic(engine, df, 'stock_basic', trade_date)
    
    print( "更新公司信息")
    df = pro.stock_company(exchange='', ts_code='')  
    sync_basic(engine, df, 'stock_company', trade_date)
    
    # print( "获得股票关键市场信息； 5000积分以上，否则每分钟最多2次, 每天20次；")
    # df = pro.bak_basic(trade_date=trade_date)
    # sync_basic(engine, df, 'bak_basic', trade_date)

    print( "更新深股通清单")
    df = pro.hs_const(hs_type='SZ')
    sync_basic(engine, df, 'sl_shengutong', trade_date)

    print("更新沪股通清单")
    df = pro.hs_const(hs_type='SH')
    sync_basic(engine, df, 'sl_hugutong', trade_date)

    print("开始改名的股票")
    df = pro.namechange(ts_code='', start_date='2019-01-01', fields='ts_code,name,start_date,end_date,change_reason')
    sync_basic(engine, df, 'namechange', trade_date)

    print("交易日历")
    df = pro.trade_cal(exchange='SSE', start_date='20190101', end_date='20221231')
    sync_basic(engine, df, 'calendar', trade_date)
    
    print("股票概念清单")   # 879+个
    df = pro.concept(src='ts') 
    sync_basic(engine, df, 'concept', trade_date)


CPU times: total: 0 ns
Wall time: 0 ns


In [6]:

# 获得股票日交易数据 - OK
def sync_tran_daily(conn, period=1):    
    now = datetime.datetime.now()
    # period = 1501
    for n in range(-period+1, 1):
        delta = datetime.timedelta(days=n)
        new_day = now + delta
        trade_date = new_day.strftime('%Y%m%d')
        print( trade_date, end="" )  

        # existDate = False
        sql="SELECT * FROM tran_daily where trade_date='" + trade_date+ "' LIMIT 1"
        df=basic.read_data(conn, sql)
        if len(df) > 0:
            print( '  - existed' )
        else:
            # read data
            df = pro.daily(trade_date=trade_date)
            df.rename( columns={'vol':'volume','pct_chg':'pct_change', 'amount':'value'}, inplace=True )
            print( '  - read: ', df.shape, end="")
            if len(df) > 0:  
                # print( '  - none ')
                # 删除已有数据
                # sql = "delete from tran_daily where trade_date="+trade_date
                # print( 'trade_date :', trade_date )
                # print( 'sql: ', sql )
                # print( 'delete res: ', basic.exec_sql(conn, sql) )
                # 添加数据
                write_no = basic.write_data(conn, df, 'tran_daily', False,'append') 
                if write_no > 0:
                    print( '  - Added', str(write_no))
                else:
                    print( '  - Added 0')
            else:    
                print( '  - none ' )
    print('\n== tran_daily: OK\n')


In [7]:
# 获得股票日资金流 - OK
def sync_moneyflow(conn, period=1):
    now = datetime.datetime.now()
    # period = 825
    for n in range(-period+1, 1):
        delta = datetime.timedelta(days=n)
        new_day = now + delta
        trade_date = new_day.strftime('%Y%m%d')
        print( trade_date, end="" )  

        # 判断当日数据是否已存在
        sql="SELECT * FROM moneyflow where trade_date='" + trade_date+ "' LIMIT 10"
        df=basic.read_data(conn, sql)
        if len(df) > 0:
            print( '  - existed' )   
        else:    
            # 读取数据
            df = pro.moneyflow(trade_date=trade_date)
            print( '  - read: ', df.shape, end="")
            if len(df) == 0:
                print( '  - no found at Tushare' )
            else:
                # 删除可能的已有数据
                # sql = "delete from moneyflow where trade_date="+trade_date
                # print( 'trade_date :', trade_date )
                # print( 'delete res: ', basic.exec_sql(conn, sql) )
                # 添加数据
                basic.write_data(conn, df, 'moneyflow', False,'append') 
                print( '  - added' )

    print('\n== moneyflow daily: OK\n')

In [9]:
%%time
"""
动态数据更新：每日成交、每日MoneyFlow
"""

# days = 365*5
# days = int(365 * 10)
days = 7

# reload(basic)

if run_dynamic:
    # 每日成交数据 -- OK
    sync_tran_daily(conn, days)
    # 每日资金流向  --  暂停
    # .sync_moneyflow(engine, days)

    # 1、国内股市：可显示复权后和复权前信息, 现在用前复权价格
    # df = ts.pro_bar(ts_code=sl_list['ts_code'][i], asset='E', adj='qfq', start_date=start_date, end_date='')

20230719  - existed
20230720  - existed
20230721  - existed
20230722  - read:  (0, 11)  - none 
20230723  - read:  (0, 11)  - none 
20230724  - read:  (5223, 11)  - Added 5223
20230725  - read:  (5226, 11)  - Added 5226

== tran_daily: OK

CPU times: total: 141 ms
Wall time: 4.55 s


In [ ]:

# import cStringIO
 
# output = cStringIO.StringIO()
# # ignore the index
# df_a.to_csv(output, sep='\t',index = False, header = False)
# output.getvalue()
# # jump to start of stream
# output.seek(0)
 
# connection = engine.raw_connection() #engine 是 from sqlalchemy import create_engine
# cursor = connection.cursor()
# # null value become ''
# cursor.copy_from(output,table_name,null='')
# connection.commit()


In [ ]:
'''管理层薪酬和持股; 需要高级权限 '''
# df = pro.stk_rewards(ts_code='601318.SH')

'''--- 新股上市清单， 120积分 '''
# df = pro.new_share(start_date='20220101', end_date='20221231')
# df.tail(10)

''' 概念股清单 
    58,TS58,锂电隔膜,ts
    59,TS59,锂电设备,ts
    60,TS60,锂电原料,ts
    61,TS61,锂电制造,ts
'''
# df = pro.concept_detail(id='TS60', fields='ts_code,name') 
# df.to_csv('~/win-desk/lidianyuanliao.txt', index=False, header=False)


' 概念股清单 \n    58,TS58,锂电隔膜,ts\n    59,TS59,锂电设备,ts\n    60,TS60,锂电原料,ts\n    61,TS61,锂电制造,ts\n'